In [1]:
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
import pandas as pd
import numpy as np
import re
import json
import numpy as np
import timeit 
from datetime import datetime

from scipy.interpolate import griddata
from numpy import linspace

In [2]:
df = pd.read_csv('data_transformed.csv')
df_clean = df.dropna(axis='rows')
df_clean

,Index,latitude,longitude,magnitude
0,0,-22.0,15.00,28.863861
1,1,-22.0,15.01,28.767861
2,2,-22.0,15.02,28.671861
3,3,-22.0,15.03,28.575862
4,4,-22.0,15.04,28.479862
...,...,...,...,...
2343000,1700,-35.0,32.00,17.463867
2343025,1725,-35.0,32.25,17.363861
2343050,1750,-35.0,32.50,17.263885
2343075,1775,-35.0,32.75,17.463867


In [3]:
df_clean = df_clean.drop('Index', axis=1)
df= df_clean
df

,latitude,longitude,magnitude
0,-22.0,15.00,28.863861
1,-22.0,15.01,28.767861
2,-22.0,15.02,28.671861
3,-22.0,15.03,28.575862
4,-22.0,15.04,28.479862
...,...,...,...
2343000,-35.0,32.00,17.463867
2343025,-35.0,32.25,17.363861
2343050,-35.0,32.50,17.263885
2343075,-35.0,32.75,17.463867


In [4]:
df = df.sample(10000)
df

,latitude,longitude,magnitude
132430,-22.73,24.57,21.784356
1253059,-28.95,28.64,5.108673
2010001,-33.16,15.85,17.185481
1502300,-30.34,17.66,22.125946
1815150,-32.07,30.43,18.524351
...,...,...,...
1381695,-29.67,18.28,18.634591
160625,-22.89,18.36,20.871871
1831664,-32.17,15.47,17.607876
1327930,-29.37,20.93,16.556193


In [5]:
center_coors=-30, 21

In [23]:
rounding_num=0.015
df["lon_mod"]=np.round(df.longitude/rounding_num,0)*rounding_num

#adjusting for difference in actual distance between same numeric change in lat and lon to create a square grid
correction_coeff=0.5
df["lat_mod"]=np.round(df.latitude/(rounding_num*correction_coeff),0)*(rounding_num*correction_coeff)

df_map=df[["lat_mod","lon_mod","magnitude"]].groupby(["lat_mod","lon_mod"], as_index=False).mean()

df_map.head()

,lat_mod,lon_mod,magnitude
0,-34.9875,15.000,16.563873
1,-34.9875,15.330,16.630597
2,-34.9875,15.780,16.779881
3,-34.9875,17.655,17.125317
4,-34.9875,19.515,16.625319


Warning! When choosing boundaries for contour plot, make sure that each layer has at least one data point. If the max and min are to wide or the step size is too small and one of the layers is empty it could make the whole geojson not appear at all in plotly.

In [6]:
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from numpy import linspace

#Extracting x,y and values (z)
z=df_map.magnitude
y=df_map.lat_mod
x=df_map.lon_mod

#Interpolating values to get better coverage
xi = linspace(x.min(),x.max(),100);
yi = linspace(y.min(),y.max(),100);
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')


#Creating contour plot with a step size of 1000
step_size=1000
cs = plt.contour(xi,yi,zi,range(5000,int( np.nanmax(zi))+step_size,step_size),cmap=plt.cm.jet)

NameError: name 'df_map' is not defined